In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Objective: Identify route and stop characteristics:

- Speed of route
- Ridership at stops
- Route spacing

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import datetime
import geopandas
import random

In [ ]:
print("GTFS data sets: ")
GTFS_DATA_PATH = "mmt_gtfs/"
import os; os.listdir(GTFS_DATA_PATH) # Peak whats in the directory 

In [ ]:
def getPandasDFCSV(path, file, sep=','):  
    """ Keyword Arg sep: deliminator used in txt file (default = ',')"""
    pandasDF = pd.read_csv(path +  file, sep=sep)
    return pandasDF

In [ ]:
stops_df = getPandasDFCSV(GTFS_DATA_PATH, 'stops.txt')
#stops_df.head()

In [ ]:
ridership_df = getPandasDFCSV(GTFS_DATA_PATH, 'Metro_Transit_Ridership_by_Route_Weekday.csv')
#ridership_df

In [ ]:
stop_times_df = getPandasDFCSV(GTFS_DATA_PATH, 'stop_times.txt')
#stop_times_df.info()
#stop_times_df.head(3)

In [ ]:
# Fix the times from strings and bad formatting to datetime objects:

arrivalTimesArray = np.array(stop_times_df['arrival_time'])

arrivalTimesArrayFixed = []
for time in arrivalTimesArray:
    timeList = time.split(":")
    timeList = [int(time) for time in timeList]
    if timeList[0] < 24:
        dateTime = datetime.datetime(2000, 1, 1, timeList[0], timeList[1], timeList[2])
    else:
        dateTime = datetime.datetime(2000, 1, 2, timeList[0]-24, timeList[1], timeList[2])
    arrivalTimesArrayFixed.append(dateTime)
    
stop_times_df['arrival_time'] = arrivalTimesArrayFixed

departureTimesArray = np.array(stop_times_df['departure_time'])

departureTimesArrayFixed = []
for time in departureTimesArray:
    timeList = time.split(":")
    timeList = [int(time) for time in timeList]
    if timeList[0] < 24:
        dateTime = datetime.datetime(2000, 1, 1, timeList[0], timeList[1], timeList[2])
    else:
        dateTime = datetime.datetime(2000, 1, 2, timeList[0]-24, timeList[1], timeList[2])
    departureTimesArrayFixed.append(dateTime)
    
stop_times_df['departure_time'] = departureTimesArrayFixed

#stop_times_df.head()

In [ ]:
#stop_times_df.head()

In [ ]:
trips_df = getPandasDFCSV(GTFS_DATA_PATH, 'trips.txt')
#trips_df.info()
#trips_df.head(3)

In [ ]:
stops_df = getPandasDFCSV(GTFS_DATA_PATH, 'stops.txt')
#stops_df.info()
#stops_df.head(3)

In [ ]:
print("list of possible trips: \n")

service_id_list = []
for i in trips_df['service_id']:
    if i not in service_id_list:
        service_id_list.append(i)
print(service_id_list)

In [ ]:
print("list of possible routes: \n")

route_short_name_list = []
for i in trips_df['route_short_name']:
    if i not in route_short_name_list:
        route_short_name_list.append(i)

route_short_name_list.sort()
print(route_short_name_list)

In [ ]:
trips_df_weekday_peak = trips_df[trips_df['service_id'] == '92_WKD']
trips_df_weekday_peak_R = trips_df[trips_df['service_id'] == '92_WKD:R']
trips_df_weekday_peak_s = trips_df[trips_df['service_id'] == '92_WKD:L#6']
trips_df_weekday_peak_s3 = trips_df[trips_df['service_id'] == '92_WKD:L=6']
trips_df_weekday_peak_s1 = trips_df[trips_df['service_id'] == '92_WKD:S']
trips_df_weekday_peak_s2 = trips_df[trips_df['service_id'] == '92_WKD:S#6']
trips_df_weekday_peak_s4 = trips_df[trips_df['service_id'] == '92_WKD:S=6']

dfs_wk = [trips_df_weekday_peak, trips_df_weekday_peak_R, trips_df_weekday_peak_s, 
          trips_df_weekday_peak_s1, trips_df_weekday_peak_s2, trips_df_weekday_peak_s4]

trips_df_weekday = pd.concat(dfs_wk)
#trips_df_weekday.info()

In [ ]:
print("Routes for weekday trips: \n")

routes_wk_list = []
for route in trips_df_weekday['route_short_name']:
    if route not in routes_wk_list:
        routes_wk_list.append(route)

routes_wk_list.sort()
print(routes_wk_list)

In [ ]:
# make a dictionary of routes with trip ID's for weekday service:
print("Dictionary of all the possible trips for each route during weekdays: \n\n Example output for 10 trips in route 2: \n")

tripIDList_forRoutes_weekday92 = dict()
for i in routes_wk_list:
    trip_list = []
    for j in trips_df_weekday[trips_df_weekday['route_short_name'] == i].iterrows():
        trip_list.append(j[1]['trip_id'])
    tripIDList_forRoutes_weekday92[i] = trip_list

print(tripIDList_forRoutes_weekday92[2][:10])

# Find average speed for each route: 

total distance travelled by route / time the route took

In [ ]:
ridership_df[ridership_df['Route'] == 7]

totalRiders = np.sum(list(ridership_df['DailyBoardings']))

In [ ]:
# Each route has a list [average speed, average stop spacing, percent of total ridership]

route_stops = dict()

totalRidersAll = np.sum(list(ridership_df['DailyBoardings']))

route_characteristics = dict()
for route in tripIDList_forRoutes_weekday92:
    route_characteristics[route] = []
    route_stops[route] = []
    
    stops_list = []
    route_trips_speeds = []
    route_trips_distances = []
    for trip in tripIDList_forRoutes_weekday92[route]:
        
        # Find speeeds: 
        stop_times_df_local = stop_times_df[stop_times_df['trip_id'] == trip]
        distance = list(stop_times_df_local['shape_dist_traveled'])[-1]
        time = list(stop_times_df_local['departure_time'])[-1] - list(stop_times_df_local['departure_time'])[0]
        speed = distance / (time.seconds / 3600)
        route_trips_speeds.append(speed)
        
        # Find average route spacing:
        spacing_list = []
        spacing = list(stop_times_df_local['shape_dist_traveled'])
        spacing.insert(0, 0.0)
        for i in range(len(spacing)-1):
            stop_spacing = spacing[i+1] - spacing[i]
            spacing_list.append(stop_spacing)
        route_trips_distances.append(np.average(spacing_list))
           
        # Add stops to dictionary:
        for stop in stop_times_df_local['stop_id']:
            if stop not in stops_list:
                stops_list.append(stop)
        
    # Get daily boardings:
    route_df = ridership_df[ridership_df['Route'] == route]
    totalRiders = np.sum(list(route_df['DailyBoardings']))
    fractionOfTotalRiders = totalRiders/totalRidersAll
    
    route_characteristics[route].append(np.average(route_trips_speeds))
    route_characteristics[route].append(np.average(route_trips_distances))
    route_characteristics[route].append(fractionOfTotalRiders)
    route_stops[route] = stops_list
    

route_characteristics[2]

In [ ]:
ax = plt.subplots(figsize=(20, 12))[1]

for i, route in enumerate(route_characteristics):
    s = route_characteristics[route]
    
    if route == 80:
        continue
    # ridership vs speed
    markerSp = "$" + str(route) + "$"
    x = s[0]
    y = s[2]
    ax.plot(x, y, color='#34013f', linewidth=3, markersize=10, marker = markerSp, alpha=.9)
    
plt.xlabel("Average Speed (MPH)")
plt.ylabel("fraction of total daily riders")
plt.title("Correlation between Ridership and Speed (Route 80 excluded)")
plt.savefig("Correlation__Ridership_Speed_no80")
plt.show()

In [ ]:
ax = plt.subplots(figsize=(20, 12))[1]

for i, route in enumerate(route_characteristics):
    s = route_characteristics[route]
    
    if route == 80:
        continue
    # ridership vs speed
    markerSp = "$" + str(route) + "$"
    x = s[1]
    y = s[2]
    ax.plot(x, y, color='#34013f', linewidth=3, markersize=10, marker = markerSp, alpha=.9)
    
plt.xlabel("Average Stop Spacing (Miles)")
plt.ylabel("fraction of total daily riders")
plt.title("Correlation between Ridership and Stop-Spacing (Route 80 excluded)")
plt.savefig("Correlation_Ridership_Stop-Spacing_no80")
plt.show()

In [ ]:
ax = plt.subplots(figsize=(20, 12))[1]

for i, route in enumerate(route_characteristics):
    s = route_characteristics[route]
    
    if route in (80, 7, 8):
        continue
    # ridership vs speed
    markerSp = "$" + str(route) + "$"
    x = s[1]
    y = s[2]
    ax.plot(x, y, color='#34013f', linewidth=3, markersize=15, marker = markerSp, alpha=.9)
    ax.set_xlabel("Average Stop Spacing (Miles)")
    ax.set_ylabel("")
    
plt.xlabel("Average Stop Spacing (Miles)", fontsize=14)
plt.ylabel("fraction of total daily riders", fontsize=14)
plt.title("Lowest ridership and closest stop spacing", fontsize=14)
plt.xlim(.16, .21)
plt.ylim(-.001, .015)
plt.savefig("Lowest_ridership_closest_stop_spacing", fontsize=18)
plt.show()

## Investigate lowest stop spacing and lowest ridership -- seems like the best for rebalancing

* Exclude UW routes: 80, 81, 82 - different purpose

focus on: 

- 36
- 32
- 44
- 27
- 51

In [ ]:
print(routes_wk_list)

In [ ]:
subRoutes = (27, 32, 44, 51)

Routes_latLon = dict()

for route in routes_wk_list:
    lat = []
    lon = []
    stops = route_stops[route]
    for stop in stops:
        stops_local = stops_df[stops_df['stop_id'] == stop]
        lat.append(float(stops_local['stop_lat']))
        lon.append(float(stops_local['stop_lon']))
    Routes_latLon[route] = [lat, lon]
    
#subRoutes_latLon[27][0][:2]

In [ ]:
riderships = []
for route in route_characteristics:
    if route == 80: 
        continue
    riderships.append(route_characteristics[route][2])

ridership_avg_no_80 = np.average(riderships)
ridership_avg_no_80

In [ ]:
Speeds = []
for route in route_characteristics:
    if route == 80: 
        continue
    Speeds.append(route_characteristics[route][0])

Speeds_avg_no_80 = np.average(Speeds)
Speeds_avg_no_80

In [ ]:
spacing = []
for route in route_characteristics:
    if route == 80: 
        continue
    spacing.append(route_characteristics[route][1])

Spacing_avg_no_80 = np.average(spacing)
Spacing_avg_no_80

In [ ]:
city = geopandas.read_file("shapes/City")
lakes = geopandas.read_file("shapes/Lakes")
Street = geopandas.read_file("shapes/Street")

In [ ]:
def plotRouteCharacteristics(route, save=True):
    ax = plt.subplots(1, 2, figsize=(20, 11), frameon=False)[1]

    ### Plot 1: Add the geopandas backgroud:
    city.plot(color="lightgray", alpha=.2, ax=ax[0], zorder=2)
    lakes.plot(color="lightblue", ax=ax[0], zorder=1, alpha=.8)
    Street.plot(color="darkgray", alpha = .5, ax=ax[0], zorder=3)

    ### Plot 2: Add the routes on top:
    x = Routes_latLon[route][1]
    y = Routes_latLon[route][0]
    
    # Look for specific colors or default to green
    try:
        ax[0].scatter(x, y, label="Route "+str(route)+" stops", color=colors[route][0], zorder=4)
    except:
        ax[0].scatter(x, y, label="Route "+str(route)+" stops", color="#2b5d34", zorder=4)
        
    ax[0].set_xlim(min(Routes_latLon[route][1])-.01, max(Routes_latLon[route][1])+.01)
    ax[0].set_ylim(min(Routes_latLon[route][0])-.005, max(Routes_latLon[route][0])+.005)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].legend()
    ax[0].set_title("Route Stops for Weekday Trips:", fontsize=16)

    ### Plot 3: Bar plot to the right summarizing stats
    ridership_perc_avg = route_characteristics[route][2]/ridership_avg_no_80
    speed_perc_avg = route_characteristics[route][0]/Speeds_avg_no_80
    spacing_perc_avg = route_characteristics[route][1]/Spacing_avg_no_80

    x_labels = ["Ridership", "Speed", "Stop Spacing"]
    hist_x = (1, 2, 3)
    width = .8
    hist_heights = (ridership_perc_avg, speed_perc_avg, spacing_perc_avg)
    
    # Look for specific colors or default to green
    try:
        ax[1].bar(x=hist_x, height=hist_heights, color=colors[route][1], tick_label=x_labels, width = width, alpha=.8)
    except:
        ax[1].bar(x=hist_x, height=hist_heights, color="#74a662", tick_label=x_labels, width = width, alpha=.8)

    # Place text above bars that too small:
    ylim = max(1, speed_perc_avg+.1, ridership_perc_avg+.1, spacing_perc_avg+.1) 
    if ridership_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("% of daily riders: \n", xy=(1, ridership_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)  
    else:
        ax[1].annotate("% of daily riders: \n", xy=(1, ridership_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if speed_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Average Speed: \n", xy=(2, speed_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Average Speed: \n", xy=(2, speed_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if spacing_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Average Spacing: \n", xy=(3, spacing_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(3, spacing_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Average Spacing: \n", xy=(3, spacing_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(3, spacing_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    ax[1].tick_params(labelsize=12)
    ax[1].set_ylabel("Fraction of Average from All Routes", fontsize=14)
    ax[1].set_ylim(0, ylim)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['top'].set_visible(False)

    if save == True:
        plt.savefig("Route_characterstics_" + str(route))
    #plt.show()
    
    return ax

In [ ]:
colors = {27: ["#2b5d34", "#74a662"], 32:["#5f1b6b", "#895b7b"], 44:["#475f94", "#77a1b5"], 51:["#730039", "#b9484e"]}

In [ ]:
plotRouteCharacteristics(27)

In [ ]:
plotRouteCharacteristics(32)

In [ ]:
plotRouteCharacteristics(44)

In [ ]:
plotRouteCharacteristics(51)

In [ ]:
plotRouteCharacteristics(2, save=False)

In [ ]:
colors_5 = {1: ["#2b5d34", "#74a662"], 2:["#5f1b6b", "#895b7b"], 3:["#475f94", "#77a1b5"], 4:["#730039", "#b9484e"], 5:["#c9643b", "#ff9408"]}

In [ ]:
def plotRouteCharacteristics_colors(route, count):
    ax = plt.subplots(1, 2, figsize=(20, 11), frameon=False)[1]

    ### Plot 1: Add the geopandas backgroud:
    city.plot(color="lightgray", alpha=.2, ax=ax[0], zorder=2)
    lakes.plot(color="lightblue", ax=ax[0], zorder=1, alpha=.8)
    Street.plot(color="darkgray", alpha = .5, ax=ax[0], zorder=3)

    ### Plot 2: Add the routes on top:
    x = Routes_latLon[route][1]
    y = Routes_latLon[route][0]
    
    # Look for specific colors or default to green
    ax[0].scatter(x, y, label="Route "+str(route)+" stops", color=colors_5[count][0], zorder=4)
        
    ax[0].set_xlim(min(Routes_latLon[route][1])-.01, max(Routes_latLon[route][1])+.01)
    ax[0].set_ylim(min(Routes_latLon[route][0])-.005, max(Routes_latLon[route][0])+.005)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].legend()
    ax[0].set_title("Route Stops for Weekday Trips:", fontsize=16)

    ### Plot 3: Bar plot to the right summarizing stats
    ridership_perc_avg = route_characteristics[route][2]/ridership_avg_no_80
    speed_perc_avg = route_characteristics[route][0]/Speeds_avg_no_80
    spacing_perc_avg = route_characteristics[route][1]/Spacing_avg_no_80

    x_labels = ["Ridership", "Speed", "Stop Spacing"]
    hist_x = (1, 2, 3)
    width = .8
    hist_heights = (ridership_perc_avg, speed_perc_avg, spacing_perc_avg)
    
    # Look for specific colors or default to green
    ax[1].bar(x=hist_x, height=hist_heights, color=colors_5[count][1], tick_label=x_labels, width = width, alpha=.8)

    # Place text above bars that too small:
    ylim = max(1, speed_perc_avg+.1, ridership_perc_avg+.1, spacing_perc_avg+.1) 
    if ridership_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("% of daily riders: \n", xy=(1, ridership_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)  
    else:
        ax[1].annotate("% of daily riders: \n", xy=(1, ridership_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if speed_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Average Speed: \n", xy=(2, speed_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Average Speed: \n", xy=(2, speed_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if spacing_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Average Spacing: \n", xy=(3, spacing_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(3, spacing_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Average Spacing: \n", xy=(3, spacing_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(3, spacing_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    ax[1].tick_params(labelsize=12)
    ax[1].set_ylabel("Fraction of Average from All Routes", fontsize=14)
    ax[1].set_ylim(0, ylim)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['top'].set_visible(False)


    plt.savefig("Route_" + str(route), bbox_inches='tight')
    plt.show()

In [ ]:
"""for i, route in enumerate(routes_wk_list):
    countNew = random.randint(1, 5)
    if i == 0:
        countOld = 0
    while(countNew == countOld):
        countNew = random.randint(1, 5)

    plotRouteCharacteristics_colors(route, countNew)
    
    countOld = countNew"""